In [41]:
import numpy as np
import pandas as pd

In [134]:
def create_mempool(txn_number, pre_df, post_df):
    mean, sd = 2, .1
    pre_gasprice = np.random.normal(mean, sd, txn_number)
    post_gasprice = np.random.normal(mean, sd, txn_number)
    values = np.random.normal(3, sd, txn_number*2)
    pre_ethvalue = [values[i]*pre_gasprice[i] for i in range(txn_number)]
    post_ethvalue = [values[i+txn_number]*post_gasprice[i] for i in range(txn_number)]

#     pre_mempool = [(pre_gasprice[i], values[i]) for i in range(txn_number)]
#     post_mempool = [(post_gasprice[i], values[i+txn_number]) for i in range(txn_number)]

    pre_df = pre_df.append(pd.DataFrame({
        'gas price': pre_gasprice,
        'value': values[:txn_number],
        'value in eth': pre_ethvalue
        }), ignore_index=True)

    post_df = post_df.append(pd.DataFrame({
        'gas price': post_gasprice,
        'value': values[txn_number:],
        'value in eth': post_ethvalue
        }), ignore_index=True)
    
    pre_df = pre_df.sort_values(by=['gas price'], ascending=False).reset_index(drop=True)
    post_df = post_df.sort_values(by=['gas price'], ascending=False).reset_index(drop=True)
    

    
    return pre_df, post_df

def pre_fill_block(pre_df):
    block = []
    block_size = 0
    for i in range(len(pre_df)):
        txn = pre_df.iloc[i, :].to_list()
        if block_size + txn[1] > 12.5:
            break
        else:
            block += [txn]
            block_size += txn[1]
    
    pre_df = pre_df.iloc[i+1:,:]    
    
    return block, pre_df
        
def simulate(n, txn_number):
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    blocks = []
    block_data = pd.DataFrame()

    pre_data, post_data = create_mempool(txn_number, df1, df2)
    
    for i in range(n):
        filled_block = pre_fill_block(pre_data)
        blocks += [filled_block[0]]
        block_data = block_data.append(pd.DataFrame({
            'number of txns': [len(filled_block[0])],
            'total gas': [sum([filled_block[0][i][1] for i in range(len(filled_block[0]))])],
            'total revenue': [sum([filled_block[0][i][2] for i in range(len(filled_block[0]))])]
        }), ignore_index=True)
        pre_data = filled_block[1]
        
        pre_data, post_data = create_mempool(txn_number, pre_data, post_data)
        
    return blocks, pre_data, block_data
    

In [135]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

s = simulate(10, 10)
print(s[2])

   number of txns  total gas  total revenue
0               4  12.055295      24.370832
1               4  11.944873      24.324579
2               4  12.034118      26.407498
3               4  11.973394      25.093937
4               4  12.271668      25.931905
5               4  12.172953      25.007433
6               4  11.983386      24.385887
7               4  11.739001      24.466607
8               4  11.812660      24.500835
9               4  11.987817      25.443000
